# Evaluating the model

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import openai

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown

In [2]:
_ = load_dotenv("config.env")
openai.api_key = os.environ['OPENAI_API_KEY']

model_name = "gpt-3.5-turbo-0301"

## Load the data

In [3]:
# Load the data
file = 'data/myntra_products_catalog.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

# Create the vector store
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    # embedding=embeddings,            # We can also manually specify the embeddings
).from_loaders([loader])

In [4]:
llm = ChatOpenAI(temperature = 0.0, model=model_name)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [5]:
data[0]

Document(page_content='ProductID: 10017413\nProductName: DKNY Unisex Black & Grey Printed Medium Trolley Bag\nProductBrand: DKNY\nGender: Unisex\nPrice (INR): 11745\nNumImages: 7\nDescription: Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer\nPrimaryColor: Black', metadata={'source': 'data/myntra_products_catalog.csv', 'row': 0})

### Generate QA examples

Instead of manually writing examples questions and reponses for the LLM, we can use the LLM itself to write these for us. Smart, right?

In [ ]:
"Does the DKNY Unisex have a trolley with a retractable handle?"